# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [41]:
import pandas as pd
import numpy as np
from scipy import stats

In [42]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [43]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [44]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

Q1.

We are testing whether the mean number of callbacks for resumes with white sounding names is equal to the mean number of callbacks for resumes with black sounding names. Therefore, I will perform a two sample z test to compare the difference in proportions. The CLT does apply here, as there are a large number of samples (over 30) and all the resumes sent out are independent of each other. 

Q2. The Null and Alternative Hypothesis:

Ho: There is no difference in the mean proportion of callbacks between resumes with black names and those with white names (P1 - P2 = 0 or P1 = P2)

Ha: There is a statistically significant difference between the mean number of callbacks between resumes with black names and those with white names (P1 - P2 != 0 or P1 != P2)

where P1 and P2 are the mean number of callbacks of the white sounding named resumes and the black soundign names resumes respectivly.

In [45]:
w = data[data.race=='w']
b = data[data.race=='b']

In [46]:
# Q3: z test, margin of error, CI, and p calculation

# I'll do the frequentist approach first:

import statsmodels.stats.weightstats as ssw

ssw.ztest(w.call, b.call, value=0, alternative='two-sided')


(4.114705356750735, 3.8767429116085706e-05)

Here we can see a zscore of 4.11 and a very small p value, indicating that we can reject the null hypothesis. Now let's do the CI and the MOE

In [47]:
w_mean = np.mean(w.call)
b_mean = np.mean(b.call)

diff_mean = w_mean = b_mean

z_95 = 1.96

MOE = z_95*np.sqrt((((w_mean*(1-w_mean))/len(w))+((b_mean*(1-b_mean))/len(b))))

print(MOE)


0.013795876753857655


So we have a margin of error of 0.01 or about 1%. Now let's calculate the CI

In [48]:
lower = diff_mean - MOE

upper = diff_mean + MOE

print('Confidence Interval: ' + str(lower) + ' ' + str(upper))

Confidence Interval: 0.05068050895876594 0.07827226246648125


So we have a Confidence Interval of (0.051, 0.078), so we can say with 95% confidence (aka 95% of all means calculated from samples from this population) that the true difference in proportions of callbacks between white sounding named resumes and black sounding named resumes is roughly between 5% and 8%. Now we're going to perform these same calculations using boostrapping using permutations

In [49]:
# First, let's define some functions that will allow us to take permutations easily:

def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1, data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2


def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates

# This next one is a function that takes the differences between means:

def diff_of_means(data_1, data_2):
    """Difference in means of two arrays."""

    # The difference of means of data_1, data_2: diff
    diff = np.mean(data_1) - np.mean(data_2)

    return diff

# now we're going to get the difference of means:

empirical_diff_means = diff_of_means(w.call, b.call)

# now we draw the permutation replicates:

perm_replicates = draw_perm_reps(w.call, b.call, diff_of_means, size=10000)

# Compute p-value: p
p = np.sum(perm_replicates >= empirical_diff_means) / len(perm_replicates)

# Print the result
print('p-value =', p)


p-value = 0.0001


From boostrapping as well, we get a very small p value. Now we're going to calculate the MOE and the confidence interval using boostrapping

In [50]:
# Now we need to define some functions that will allow us to draw boostrap replicates:

def bootstrap_replicate_1d(data, func):
     """Generate bootstrap replicate of 1D data."""
     bs_sample = np.random.choice(data, len(data))
     return func(bs_sample)

def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data, func)

    return bs_replicates

# Compute the difference of the sample means: mean_diff
mean_diff =  diff_of_means(w.call, b.call)

# Get bootstrap replicates of means
bs_replicates_b = draw_bs_reps(b.call, np.mean, 10000)
bs_replicates_w = draw_bs_reps(w.call, np.mean, 10000)

# Compute samples of difference of means: bs_diff_replicates
bs_diff_replicates = bs_replicates_w - bs_replicates_b

# Compute 95% confidence interval: conf_int
conf_int = np.percentile(bs_diff_replicates, [2.5, 97.5])

# Print the results
print('difference of means =', mean_diff, 'mm')
print('95% confidence interval =', conf_int, 'mm')

difference of means = 0.03203285485506058 mm
95% confidence interval = [0.01683778 0.04722793] mm


Here we can see a different confidence interval from the one calculated using bayesian statistics. We have a CI of roughly (0.017, 0.047), so the difference between these two proportions is estimated to be between about 2% and 5%, versus the 5% and 8% in the previous CI. The CI here don't even overlap. 

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

Q4. 

Our tests have concluded that there is a statistically significant difference in the proportion of callbacks resumes with white sounding names get versus the proportion of callbacks resumes with black sounding names revieve. Even though our confidence intervals varied (5%-8% CI VS 2%-5%), neither of our CIs contained 0, and the p-values calculated from both approaches were very small (close to 0). This implies that there is a statistically significant difference in the proportion of callbacks between these two racial groups. The small p value indicates that this difference is significant at the 99% level.

Q5. 

While this analysis does show that race is a factor in whether or not a resume gets a callback, we do not have enough evidence to support the idea that it is the most important factor. There are many other factors in this dataset, such as experience and number of jobs, which could also play a role in determining whether a resume gets a callback. If I wanted to determine the most important factor, I would have to create a predictive model, with the liklihood of a callback as my dependent variable, and all the other factors as my independent variables. I would then remove the facots that had a high p value and don't make a statistically significant difference in the likelihood of getting a callback. I would then order the remaining variables by effect size (or correlation (r)) and the variable with the largest effect size would be considered the most important factor. This may be race, or it may be some other variable. This analysis would determine which variables contributed most to the decision of whetehr or not a particular resume gets a callback.